In [1]:
#cnn convolution neural network

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
num_classes = 10 #분류종류 10개
learning_rate = 0.001 #hyper parameter
training_steps = 200 #epoch(한번에 로드되는 데이터의 갯수)
batch_size = 128
display_step = 10

conv1_filters = 32 #학습대상 - 하나의 이미지에서 32개의 특징을 추출한다.
conv2_filters = 64 
fc1_units = 1024

In [4]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train/255., x_test/255.

In [5]:
train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [6]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)) #from_tensor_slices - 일정한 사이즈로 데이터를 쪼갠다.

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME') 
    #strides - 건너뛰는 갯수, padding - 입력차수, 출력차수 동일
    x = tf.nn.bias_add(x,b) #y = ax+b (bias_add - 바이어스의 상,하한치를 고려해준다.)
    return tf.nn.relu(x)
def maxpool2d(x, k=2):
    #ksize - 커널 사이즈 (1/4로 줄어든다.)
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

In [ ]:
random_normal = tf.initializers.RandomNormal() #함수 인스턴스
weights = {
    #5*5 = filter size, 1->입력채널수, conv1_filters->출력차수
    'wc1' : tf.Variable(random_normal([5,5,1,conv1_filters])),
    'wc2' : tf.Variable(random_normal([5,5,conv1_filters, conv2_filters])),
    #wd = dense망의 가중치
    'wd1' : tf.Variable(random_normal([7*7*64, fc1_units])),
    'out' : tf.Variable(random_normal([fc1_units, num_classes]))
}
biases = {
    #나가는 차수에 맞춰서 생성된다.
    'bc1' : tf.Variable(tf.zeros([conv1_filters])),
    'bc2' : tf.Variable(tf.zeros([conv2_filters])),
    'bd1' : tf.Variable(tf.zeros([fc1_units])),
    'out' : tf.Variable(tf.zeros([num_classes]))
}

In [ ]:
def conv_net(x):
    x = tf.reshape(x, [-1, 28, 28, 1]) #128*28*28*1
    conv1 = conv2d(x, weights['wc1'], biases['bc1']) #128*28*28*32
    conv1 = maxpool2d(conv1, k=2) #128*14*14*32
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2']) #128*14*14*64
    conv2 = maxpool2d(conv2, k=2) #128*7*7*64
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]]) #128*1024
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out']) #128*10
    return tf.nn.softmax(out)

In [ ]:
def cross_entropy(y_pred, y_true):
    #예측값과 실제값
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.) #clip_by_value - 값에 제한을 주는것
    return tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred)))
def accuracy(y_pred, y_true):
    correct_prediction=tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64)) #equal 비교연산자 결과값 비교(boolean값으로 리턴)
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)
optimizer = tf.optimizers.Adam(learning_rate)

In [ ]:
def run_optimization(x,y):
    with tf.GradientTape() as g:
        #미분에 대상이 되는값을 저장한다.
        pred = conv_net(x) #통과하면 분류예측값이 된다.(0~1사이의 확률값으로 나옴)
        loss = cross_entropy(pred, y) #loss값 비용값 산정
    train_variables = list(weights.values())+list(biases.values())
    grdients = g.gradient(loss, train_variables) #학습의 대상이 된다. (편미분)
    #여러개의 변수로 구성된 식에서 하나면 변수로 보고 나머지는 상수로 취급
    optimizer.apply_gradients(zip(grdients, train_variables))

In [ ]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y) #함수가 계층적으로 호출(내부적으로는 그래프를 만들어서 실행된다.)
    if step % display_step == 0:
        #배수를 찾아서 출력해라
        pred = conv_net(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print('step :  %i, loss : %f, accuracy : %f'%(step, loss, acc))

In [ ]:
#문제 :테스트 accuracy 를 출력하시오
    #test 데이터 5장을 출력하고 예측을 출력하시오

In [ ]:
pred = conv_net(x_test)
acc = accuracy(pred, y_test)

In [ ]:
import matplotlib.pyplot as plt
n_image = 5
test_image = x_test[:n_image]
pred = conv_net(test_image)

for i in range(n_image):
    plt.imshow(np.reshape(test_image[i], [28,28]), cmap='gray')
    plt.show()
    print('모델예측결과 : %i'%(tf.argmax(pred[i]).numpy()))

In [ ]:
#keras의 모델을 상속받아서 구현해보아라.

In [ ]:
from tensorflow.keras import Model, layers

class ConvNet(Model):
    #원래 생성자 호출은 불가능한데 예외가 있다 상속받은 경우에는 부모의 생성자를 직접 호출할 수 있다.
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = layers.Conv2D(32, kernel_size=5, activation=tf.nn.relu)
        self.maxpool1 = layers.MaxPool2D(2, strides=2)
        self.conv2 = layers.Conv2D(64, kernel_size=3, activation=tf.nn.relu)
        self.maxpool2 = layers.MaxPool2D(2, strides=2)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(1024)
        self.dropout = layers.Dropout(rate = 0.5)
        self.out = layers.Dense(num_classes)

    def call(self, x, is_training = False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=is_training)
        x = self.out(x)
        if not is_training:
            x=tf.nn.softmax(x)
        return x
conv_net = ConvNet()

In [ ]:
def cross_entropy_loss(x,y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)
optimizer = tf.optimizers.Adam(learning_rate)

In [ ]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = conv_net(x, is_training =True)
        loss = cross_entropy_loss(pred, y)
    trainable_variables = conv_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps),1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0 :
        pred = conv_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print('step: %i, loss: %f, accuracy: %f'%(step, loss, acc))

In [76]:
#tensorboard

In [77]:
logs_path = '/tmp/logs/'
num_classes = 10
num_features = 784
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100
n_hidden_1 = 128
n_hidden_2 = 256

In [78]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train/255., x_test/255.

In [79]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [80]:
random_normal = tf.initializers.RandomNormal()
weights={
    'h1_weights' : tf.Variable(random_normal([num_features, n_hidden_1]), name='h1_weights'),
    'h2_weights' : tf.Variable(random_normal([n_hidden_1, n_hidden_2]), name='h2_weights'),
    'logits_weights' : tf.Variable(random_normal([n_hidden_2, num_classes]), name='logits_weights')
}
biases={
    'h1_bias' : tf.Variable(tf.zeros([n_hidden_1]), name='h1_bias'),
    'h2_bias' : tf.Variable(tf.zeros([n_hidden_2]), name='h2_bias'),
    'logits_bias' : tf.Variable(tf.zeros([num_classes]), name='logits_bias')
}

In [118]:
#@tf.function
def neural_net(x):
    with tf.name_scope('Model'):
        with tf.name_scope('Hiddenlayer1'):
            layer1 = tf.add(tf.matmul(x,weights['h1_weights']), biases['h1_bias'])
            layer_1 = tf.nn.sigmoid(layer1)
        with tf.name_scope('Hiddenlayer2'):
            layer2 = tf.add(tf.matmul(layer1,weights['h2_weights']), biases['h2_bias'])
            layer_2 = tf.nn.sigmoid(layer2)
        with tf.name_scope('Logitslayer'):
            out_layer = tf.matmul(layer_2, weights['logits_weights'])+biases['logits_bias']
            out_layer = tf.nn.softmax(out_layer)
    return out_layer

In [119]:
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred)))
def accuracy(y_pred, y_true):
    with tf.name_scope('Accuracy'):
        correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
        return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)
with tf.name_scope('Optimizer'):
    optimizer = tf.optimizers.SGD(learning_rate)

In [120]:
def run_optimizatoin(x,y):
    with tf.GradientTape() as g:
        pred = neural_net(x)
        loss = cross_entropy(pred, y)
    trainable_variables = list(weigths.values())+list(biases.values())
    gradient = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradient, trainable_variables))

In [121]:
def summarize_weights(step):
    for w in weights:
        #히스토그램을 그릴수 있도록 출력하라.
        #replace('_','/') - 저장할때 경로 지정 _를 /로 바꿔주었다.
        tf.summary.histogram(w.replace('_','/'), weights[w], step=step)
    for b in biases:
        tf.summary.histogram(b.replace('_','/'), biases[b], step=step)

In [122]:
summary_writer = tf.summary.create_file_writer(logs_path)
#인스턴스해서 경로를 부여해야한다.
#tensorboard --logdir=/tmp/logs 
#summary는 시각화에 사용한다.

In [123]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps),1):
    if step==1:
        #step 1일때 시각화모드를 킨다.
        #profiler=True - 그때그때 값을 확인할 때 사용
        tf.summary.trace_on(graph=True, profiler=True)
    run_optimization(batch_x, batch_y)
    if step==1:
        #trace_export - 출력시작메시지
        with summary_writer.as_default():
            tf.summary.trace_export(name='trace', step=0, profiler_outdir=logs_path)
    if step % display_step == 0:
        pred = neural_net(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print('step :%i, loss :%f, accuracy :%f'%(step, loss, acc))
        #그래프나 summary_write는 default만 작동한다.
        #여러개를 각각 만들수 있다. 현재작동하는것은 항상 as_deault()
        with summary_writer.as_default():
            tf.summary.scalar('loss',loss, step=step) #scalar값으로 저장(단일값)
            tf.summary.scalar('accuracy', acc, step=step)
            summarize_weights(step) #가중치 저장

step :100, loss :610.747803, accuracy :0.078125
step :200, loss :614.890991, accuracy :0.113281
step :300, loss :629.696411, accuracy :0.085938
step :400, loss :615.283142, accuracy :0.117188
step :500, loss :608.083679, accuracy :0.097656
step :600, loss :610.698120, accuracy :0.109375
step :700, loss :621.163086, accuracy :0.082031
step :800, loss :613.957581, accuracy :0.074219
step :900, loss :610.453491, accuracy :0.093750
step :1000, loss :614.234680, accuracy :0.082031
step :1100, loss :600.497986, accuracy :0.085938
step :1200, loss :616.836060, accuracy :0.097656
step :1300, loss :618.177612, accuracy :0.093750
step :1400, loss :600.834229, accuracy :0.101562
step :1500, loss :615.051086, accuracy :0.089844
step :1600, loss :603.918701, accuracy :0.121094
step :1700, loss :603.460205, accuracy :0.109375
step :1800, loss :604.547058, accuracy :0.125000
step :1900, loss :615.200500, accuracy :0.101562
step :2000, loss :604.346497, accuracy :0.105469
step :2100, loss :595.655884,